<a href="https://colab.research.google.com/github/RutujKhare1/CS6910_Assignment3/blob/main/FDL_A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import wandb

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
import pandas as pd
import torch
import random
import torch.nn as nn

In [2]:
import torch
if torch.cuda.is_available():
    device_name = torch.device("cuda")
else:
    device_name = torch.device('cpu')
print("Using {}.".format(device_name))

Using cuda.


In [3]:
df = pd.read_csv('/content/drive/MyDrive/FDL_A3/hin/hin_train.csv', names=['eng','hin'])

In [4]:
eng_words = df['eng'].copy()
eng_maxlen = len(max(df['eng'], key=len))
for i in range(len(eng_words)):
  l = len(eng_words[i])
  eng_words[i] = eng_words[i] + "*"*(eng_maxlen - l + 2)

In [5]:
hin_words = df['hin'].copy()
hin_maxlen = len(max(df['hin'], key=len))
for i in range(len(hin_words)):
  l = len(hin_words[i])
  hin_words[i] = "#" + hin_words[i] + "*"*(hin_maxlen - l + 2)

In [6]:
unique_eng_letters = set(''.join(eng_words))
unique_hin_letters = set(''.join(hin_words))
int_to_eng = dict(enumerate(unique_eng_letters))
eng_to_int = {char: ind for ind, char in int_to_eng.items()}

int_to_hin = dict(enumerate(unique_hin_letters))
hin_to_int = {char: ind for ind, char in int_to_hin.items()}

In [7]:
index_eng_words = []
for eng_word in eng_words:
  index_eng_word = [eng_to_int[i] for i in eng_word]
  index_eng_words.append(index_eng_word)

In [8]:
index_hin_words = []
for hin_word in hin_words:
  index_hin_word = [hin_to_int[i] for i in hin_word]
  index_hin_words.append(index_hin_word)

In [9]:
tensor_eng = torch.tensor(index_eng_words).to(device_name)
tensor_hin = torch.tensor(index_hin_words).to(device_name)
tensor_hin

tensor([[62, 57, 14,  ..., 40, 40, 40],
        [62, 17, 42,  ..., 40, 40, 40],
        [62, 23, 42,  ..., 40, 40, 40],
        ...,
        [62, 21, 14,  ..., 40, 40, 40],
        [62, 14, 65,  ..., 40, 40, 40],
        [62, 21, 39,  ..., 40, 40, 40]], device='cuda:0')

In [10]:
class GRU_Encoder(nn.Module):
  def __init__(self, input_size, hid_size, num_of_enc_layers, emb_size, batch_size, bi_direct):
    super(GRU_Encoder, self).__init__()
    self.input_size = input_size
    self.hid_size = hid_size
    self.num_of_enc_layers = num_of_enc_layers
    self.emb_size = emb_size
    self.batch_size = batch_size
    self.bi_direct = bi_direct
    self.embedding = nn.Embedding(input_size, emb_size)
    # print("IS:{} ES:{}".format(input_size, emb_size))
    self.gru = nn.GRU(emb_size, hid_size, num_of_enc_layers, bidirectional = bi_direct)

  def forward(self, input_data, hidden):
    # print(input_data.shape)
    embed = self.embedding(input_data).to(device_name)
    # print(embed.shape)
    embed = embed.view(-1, self.batch_size, self.hid_size)
    output, hidden = self.gru(embed, hidden)
    if(self.bi_direct):
      hidden = hidden.resize(2, self.num_of_enc_layers, self.batch_size, self.hid_size)
      hidden = torch.add(hidden[0], hidden[1])/2
    return output, hidden
  
  def initialiseHidden(self):
    if(self.bi_direct):
      return torch.zeros(2*self.num_of_enc_layers, self.batch_size, self.hid_size, device = device_name)
    else:
      return torch.zeros(self.num_of_enc_layers, self.batch_size, self.hid_size, device = device_name)
  

In [11]:
class GRU_Decoder(nn.Module):
  def __init__(self, op_size, num_of_dec_layers, hid_size, batch_size, emb_size, bi_direct):
    super(GRU_Decoder, self).__init__()
    self.op_size = op_size
    self.hid_size = hid_size
    self.num_of_dec_layers = num_of_dec_layers
    self.emb_size = emb_size
    self.batch_size = batch_size
    self.bi_direct = bi_direct
    self.embedding = nn.Embedding(op_size, emb_size)
    self.op = nn.Linear(hid_size, op_size)
    self.softmax = nn.LogSoftmax(dim = 1)
    self.gru = nn.GRU(emb_size, hid_size, num_of_dec_layers, bidirectional = bi_direct)

  def forward(self, input_data, hidden):
    # print(input_data)
    embed = self.embedding(input_data)
    embed = embed.view(-1, self.batch_size, self.emb_size)
    out, hidden = self.gru(embed, hidden)
    out = self.softmax(self.op(out))
    return out, hidden

In [19]:
def train(input_data, target_data, loss_fn, enc_optimizer, dec_optimizer, encoder, decoder, num_of_enc_layers, num_of_dec_layers, batch_size, cell_type):
  teacher_forcing = 0.5
  loss = 0
  for b in range(0, len(input_data), batch_size):
    x, y = input_data[b : b+batch_size], target_data[b : b+batch_size]
    temp = 0
    enc_optimizer.zero_grad()
    dec_optimizer.zero_grad()
    # x = x.T
    # y = y.T
    t_step = len(x)
    if(cell_type == 'GRU'):
      enc_hidden = encoder.initialiseHidden()
      enc_output, enc_hidden = encoder(x, enc_hidden)
      
      if(num_of_dec_layers > num_of_enc_layers):
        num = num_of_dec_layers
        dec_hidden = enc_hidden
        while(num != num_of_enc_layers):
          dec_hidden = torch.cat([dec_hidden, enc_hidden[-1].unsqueeze(0)], dim = 0)
          num -= 1
      elif(num_of_dec_layers < num_of_enc_layers):
        dec_hidden = enc_hidden[-num_of_dec_layers:]
      else:
        dec_hidden = enc_hidden
      y = y.T
      dec_input = y[0]
      condition = False if random.random() > teacher_forcing else True
      if(condition):
        for i in range(1,len(y)):
          dec_output, dec_hidden = decoder(dec_input, dec_hidden)
          temp += loss_fn(torch.squeeze(dec_output), y[i])
          dec_input = y[i]
      else:
        for i in range(1,len(y)):
          dec_output, dec_hidden = decoder(dec_input, dec_hidden)
          prob, idx = dec_output.topk(1)
          temp += loss_fn(torch.squeeze(dec_output), y[i])
          dec_input = idx
    temp.backward()
    enc_optimizer.step()
    dec_optimizer.step()
    loss += temp
  # print(len(target_data))
  return loss.item() / len(target_data), encoder, decoder



In [20]:
def eval(input_data, target_data, encoder, decoder, num_of_enc_layers, num_of_dec_layers, cell_type):
  x, y = input_data, target_data
  encoder.eval()
  decoder.eval()
  predicted_data = list()
  # x = x.T
  # y = y.T
  t_step = len(x)
  if(cell_type == 'GRU'):
    enc_hidden = encoder.initialiseHidden()
    enc_output, enc_hidden = encoder(x, enc_hidden)
    
    if(num_of_dec_layers > num_of_enc_layers):
      num = num_of_dec_layers
      dec_hidden = enc_hidden
      while(num != num_of_enc_layers):
        dec_hidden = torch.cat([dec_hidden, enc_hidden[-1].unsqueeze(0)], dim = 0)
        num -= 1
    elif(num_of_dec_layers < num_of_enc_layers):
      dec_hidden = enc_hidden[-num_of_dec_layers:]
    else:
      dec_hidden = enc_hidden
    y = y.T
    dec_input = y[0]

    for i in range(len(y)):
      dec_output, dec_hidden = decoder(dec_input, dec_hidden)
      prob, idx = dec_output.topk(1)
      idx = idx.squeeze()
      dec_input = idx
      predicted_data.append(idx.tolist())
  return predicted_data


In [21]:
def training(input_data, input_size, target_data, target_size, max_input_size, epochs, batch_size, emb_size, num_of_enc_layers, num_of_dec_layers, hid_size, cell_type, bi_direct, dropout, beam_size):
  learning_rate = 0.001
  if(cell_type == "GRU"):
    encoder = GRU_Encoder(input_size, hid_size, num_of_enc_layers, emb_size, batch_size, bi_direct).to(device_name)
    decoder = GRU_Decoder(target_size, num_of_dec_layers, hid_size, batch_size, emb_size, bi_direct).to(device_name)
  
  enc_optimizer = torch.optim.Adam(encoder.parameters(), learning_rate)
  dec_optimizer = torch.optim.Adam(decoder.parameters(), learning_rate)
  loss_fn = nn.CrossEntropyLoss(reduction = 'sum')
  encoder.train()
  decoder.train()
  loss_list = []
  for i in range(epochs):
    loss, encoder, decoder = train(input_data, target_data, loss_fn, enc_optimizer, dec_optimizer, encoder, decoder, num_of_enc_layers, num_of_dec_layers, batch_size, cell_type)
    loss_list.append(loss/51200)
    print(loss)

  return encoder, decoder, num_of_enc_layers, num_of_dec_layers


In [46]:
encoder, decoder, num_of_enc_layers, num_of_dec_layers = training(input_data = tensor_eng, 
                                                                  input_size = len(unique_eng_letters), 
                                                                  target_data = tensor_hin, 
                                                                  target_size = len(unique_hin_letters), 
                                                                  max_input_size = 24, 
                                                                  epochs = 10, 
                                                                  batch_size = 32, 
                                                                  emb_size = 64, 
                                                                  num_of_enc_layers = 2, 
                                                                  num_of_dec_layers = 2, 
                                                                  hid_size = 64, 
                                                                  cell_type = "GRU", 
                                                                  bi_direct = False, 
                                                                  dropout = 0.2, 
                                                                  beam_size = 2)

51200
39.156083984375
51200
32.2370068359375
51200
31.3527392578125
51200
31.0369921875
51200
30.93255615234375
51200
30.706103515625
51200
30.4196826171875
51200
30.24394775390625
51200
30.11994873046875
51200
30.1342431640625


In [47]:
trained_pred = eval(tensor_eng, tensor_hin, encoder, decoder, num_of_enc_layers, num_of_dec_layers, "GRU")

In [48]:
temp = torch.tensor(trained_pred)
temp = temp.view(23, 51200)
pred_up = temp.T.to(device_name)

In [43]:
def calculateAccuracy(y_pred, y_true):
  correct = torch.eq(y_pred, y_true)

  # calculate accuracy
  accuracy = torch.mean(correct.float())

  print('Accuracy:', accuracy.item())

In [44]:
calculateAccuracy(pred_up, tensor_hin)

Accuracy: 0.16736668348312378


In [19]:
df_test = pd.read_csv('/content/drive/MyDrive/FDL_A3/hin/hin_test.csv', names=['eng','hin'])
eng_words = df_test['eng']
maxlen = len(max(df['eng'], key=len))
index_eng_words = []
for eng_word in eng_words:
  index_eng_word = [eng_to_int[i] for i in eng_word]
  l = len(index_eng_word)
  index_eng_word.extend([0]*(maxlen-l+2))
  index_eng_words.append(index_eng_word)

tensor_eng_test = torch.tensor(index_eng_words).to(device_name)
tensor_hin_test = torch.tensor(index_hin_words).to(device_name)

In [20]:
pred_value = eval(tensor_eng_test, tensor_hin_test, )

TypeError: ignored

# **Experimental Area**

In [ ]:
maxlen = len(max(df['eng'], key=len))

In [ ]:
len(tensor_eng)

In [ ]:
for i in df['eng']:
  if(len(i) > 24):
    print(i)

In [ ]:
a = [1,2,3,4]
a.append(0*4)
a

In [ ]:
random.random()

In [ ]:
for x, y in zip(tensor_eng, tensor_hin):
  print(x,y)

torch.Size([26, 66])
torch.Size([23])


ValueError: ignored